In [1]:
import wandb
from youtube import download_youtube_audio

In [2]:
url = "https://www.youtube.com/watch?v=-kFOXP026eE&list=PLS3_1JNX8dEh5YcO-Y05stU0u_T9nqIlF&index=1"

In [3]:
audio_file = download_youtube_audio(url)
audio_file

'/home/tcapelle/wandb/whisper/./richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4'

In [4]:
audio_file

'/home/tcapelle/wandb/whisper/./richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4'

In [19]:
with wandb.init(project="whisper", job_type="log_artifact"):
    at = wandb.Artifact("audio_file", type="audio", description="Richard Feymann Lectures at Cornell: Part 1", 
                        metadata={"url":url})
    at.add_file(audio_file)
    wandb.log_artifact(at)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


## Whisper

In [1]:
import torch
import wandb
import whisper
from whisper.transcribe import write_vtt
from types import SimpleNamespace

from pathlib import Path

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "tiny.en"
artifact = 'capecape/whisper/audio_file:v0'
url = "https://www.youtube.com/watch?v=-kFOXP026eE&list=PLS3_1JNX8dEh5YcO-Y05stU0u_T9nqIlF&index=1"
fname = "richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4"

In [5]:
def transcribe_file(model, file, log=True, language="en"):
    "Run whisper on an audio file and save results"
    print(f"Transcribing file: {file}")
    file = Path(file)
    result = model.transcribe(str(file), verbose=False, log=log, language=language)

    # save TXT
    with open(file.with_suffix(".txt"), "w", encoding="utf-8") as txt:
        print(result["text"], file=txt)

    # save VTT
    with open(file.with_suffix(".vtt"), "w", encoding="utf-8") as vtt:
        write_vtt(result["segments"], file=vtt)
    return result

In [ ]:
for model_name in "tiny.en,small.en,base.en,medium.en".split(","):
    model = whisper.load_model(model_name).to(device)
    config = SimpleNamespace(model=model_name, device=device, url=url, fname=fname, artifact=artifact)    
    with wandb.init(project="whisper", job_type="bench", config=config):
        artifact = wandb.use_artifact('capecape/whisper/audio_file:v0', type='audio')
        audio_file = Path(artifact.download())/fname
        res = transcribe_file(model, audio_file)

wandb: Downloading large artifact audio_file:v0, 50.95MB. 1 files... Done. 0:0:0.0


Transcribing file: artifacts/audio_file:v0/richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336401/336401 [01:41<00:00, 3305.10frames/s]


frames_s,▂▁▄▃▃▃▃▃▂▃▃▃▃▂▂▃▂▃▃▂▂▅▃▂▂▃▃▃▃▃▃▃▂▂▃▃▃▂▃█
frames_s,13357.09067


wandb: Downloading large artifact audio_file:v0, 50.95MB. 1 files... Done. 0:0:0.0


Transcribing file: artifacts/audio_file:v0/richard_feynman_messenger_lectures_at_cornell__the_character_of_physical_law__part_1_the_law_of_gravitation.mp4


 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                          | 308284/336401 [03:35<00:20, 1390.29frames/s]